## COMPUTATION OF BETTI NUMBERS FOR A GIVEN SIMPLICIAL COMPLEX

---

(a) Implement the Incremental Algorithm (8) and Gauss Reduction
Algorithm (9) in the lecture note: Incremental Algorithm. The input
should be a simplicial complex K (as a list or dictionary) or a filtra-
tion of subcomplexes K1 ⊂ K2 ⊂ K3 ⊂ · · · ⊂ Kn = K.

(b) You can simply use filtration by the dimension of the simplices.

(c) The output should be a list consisting of Betti numbers β0(K), β1(K), . . . , βdim(K)(K).

(d) Test your algorithm on triangulations of the following spaces:

(i) A circle,

(ii) A 2 dimensional sphere ,

(iii) A torus

---

### Summary of Incremental Algorithm for Computing Betti Numbers

This Python code calculates Betti numbers for a simplicial complex using boundary matrices and reduction techniques.

#### Key Functions

1. **Boundary Matrix**:
   - Constructs a boundary matrix indicating which simplices are faces of others.

2. **Delta Calculation**:
   - Computes a delta vector to track the largest index of non-zero entries in each column of the boundary matrix.

3. **Matrix Reduction**:
   - Performs Gaussian reduction on the boundary matrix, combining columns with the same delta value until no further updates are possible.

4. **Positivity Check**:
   - Checks if a column in the reduced matrix contains any positive entries.

5. **Betti Number Calculation**:
   - Computes and update the Betti numbers based on the reduced matrix's properties.

### Output
- Returns a list of Betti numbers, which indicate the connectivity of the simplicial complex.



In [1]:
import gudhi 
import numpy as np
import networkx as nx
import matplotlib.pyplot as plt
import itertools

---

In [2]:
import numpy as np
from typing import List, Tuple

# Function to compute the boundary matrix
def boundary_matrix(simpcomplex):
    num_simplices = len(simpcomplex)
    b_matrix = np.zeros((num_simplices, num_simplices), dtype=int)

    for i, simplex_i in enumerate(simpcomplex):
        for j, simplex_j in enumerate(simpcomplex):
            # Check if simplex_i is a face of simplex_j
            if len(simplex_i) + 1 == len(simplex_j) and set(simplex_i).issubset(simplex_j):
                b_matrix[i, j] = 1

    return b_matrix


# Function to compute the gauss reduction coefficient
def calculate_delta(b_matrix):
    num_columns = b_matrix.shape[1]
    delta = np.zeros(num_columns, dtype=int)
    for j in range(num_columns):
        for i in range(num_columns):
            if b_matrix[i, j] != 0:
                delta[j] = i  # Update to the largest index with a non-zero value
    return delta

# Function to reduce the boundary matrix
def reduce_matrix(b_matrix):
    num_columns = b_matrix.shape[1]
    delta = calculate_delta(b_matrix)  # Initial delta calculation
    print("Initial Delta:\n", delta)

    while True:
        updated = False
        for j in range(num_columns):
            for i in range(j):
                # Check if both delta values are defined and equal
                if delta[i] == delta[j] and delta[j] != 0:
                    b_matrix[:, j] = (b_matrix[:, j] + b_matrix[:, i]) % 2
                    delta = calculate_delta(b_matrix)  # Update delta after modification
                    updated = True
                    print(f"Updated column {j} with column {i}. New Delta:\n", delta)
                    break  # Break to restart checking from the beginning
            if updated:
                break  # Restart outer loop if an update happened
        if not updated:
            break  # Exit if no updates were made

    print("Final Delta:\n", delta)
    return b_matrix

# Function to identify if a simplex is positive or not
def is_positive(red_matrix, index):
    return np.any(red_matrix[:, index] > 0)


# Function to compute the betti numbers
def incremental_algorithm(simplicial_complexes: List[Tuple[int, ...]]) -> List[int]:
    b_matrix = boundary_matrix(simplicial_complexes)
    print("Boundary Matrix:\n", b_matrix)
    
    red_matrix = reduce_matrix(b_matrix.copy())
    print("Reduced Matrix:\n", red_matrix)

    n = max(len(simplex) - 1 for simplex in simplicial_complexes)
    
    # Initialize Betti numbers list
    betti = [0] * (n + 1)
    for i, simplex in enumerate(simplicial_complexes):
        dim = len(simplex) - 1
        if not is_positive(red_matrix, i):
            betti[dim] += 1
        else:
            betti[dim - 1] -= 1
    return betti

---

---

### Application the example of the course

In [5]:
# Define your simplicial complex
simpcomplex2 = [  
    (0,),  
    (1,),  
    (2,),  
    (3,),  
    (0, 1),  
    (1, 2),  
    (1, 3),  
    (2, 3),  
    (0, 3),  
    (0, 1, 3)  
]

# Compute the boundary matrix
betti = incremental_algorithm(simpcomplex2)
print("Betti numbers:", betti)

Boundary Matrix:
 [[0 0 0 0 1 0 0 0 1 0]
 [0 0 0 0 1 1 1 0 0 0]
 [0 0 0 0 0 1 0 1 0 0]
 [0 0 0 0 0 0 1 1 1 0]
 [0 0 0 0 0 0 0 0 0 1]
 [0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 1]
 [0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 1]
 [0 0 0 0 0 0 0 0 0 0]]
Initial Delta:
 [0 0 0 0 1 2 3 3 3 8]
Updated column 7 with column 6. New Delta:
 [0 0 0 0 1 2 3 2 3 8]
Updated column 7 with column 5. New Delta:
 [0 0 0 0 1 2 3 0 3 8]
Updated column 8 with column 6. New Delta:
 [0 0 0 0 1 2 3 0 1 8]
Updated column 8 with column 4. New Delta:
 [0 0 0 0 1 2 3 0 0 8]
Final Delta:
 [0 0 0 0 1 2 3 0 0 8]
Reduced Matrix:
 [[0 0 0 0 1 0 0 0 0 0]
 [0 0 0 0 1 1 1 0 0 0]
 [0 0 0 0 0 1 0 0 0 0]
 [0 0 0 0 0 0 1 0 0 0]
 [0 0 0 0 0 0 0 0 0 1]
 [0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 1]
 [0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 1]
 [0 0 0 0 0 0 0 0 0 0]]
Betti numbers: [1, 1, 0]


---

### Application on Circle

In [3]:
# Example usage
simplicial_complexes = [
    # Define your simplicial complexes here, as a list of lists or similar structure
    
        (0,),         # 0-simplex
        (1,),         # 1-simplex
        (2,),         # 1-simplex
        (0, 1),       # 2-simplex
        (1, 2),       # 2-simplex
        (0, 2),       # 2-simplex
        #(0, 1, 2)     # 2-simplex
        # (0, 1, 2, 3) # 3-simplex, if needed
    
]

betti = incremental_algorithm(simplicial_complexes)
print("Betti numbers:", betti)

Boundary Matrix:
 [[0 0 0 1 0 1]
 [0 0 0 1 1 0]
 [0 0 0 0 1 1]
 [0 0 0 0 0 0]
 [0 0 0 0 0 0]
 [0 0 0 0 0 0]]
Initial Delta:
 [0 0 0 1 2 2]
Updated column 5 with column 4. New Delta:
 [0 0 0 1 2 1]
Updated column 5 with column 3. New Delta:
 [0 0 0 1 2 0]
Final Delta:
 [0 0 0 1 2 0]
Reduced Matrix:
 [[0 0 0 1 0 0]
 [0 0 0 1 1 0]
 [0 0 0 0 1 0]
 [0 0 0 0 0 0]
 [0 0 0 0 0 0]
 [0 0 0 0 0 0]]
Betti numbers: [1, 1]


## Application on Sphere

In [6]:
# Example triangulation of a sphere (tetrahedron as a simple case)
simplicial_complexes = [
    (0,),         # 0-simplex (vertex)
    (1,),         # 1-simplex (vertex)
    (2,),         # 1-simplex (vertex)
    (3,),         # 1-simplex (vertex)
    (0, 1),       # 2-simplex (edge)
    (0, 2),       # 2-simplex (edge)
    (0, 3),       # 2-simplex (edge)
    (1, 2),       # 2-simplex (edge)
    (1, 3),       # 2-simplex (edge)
    (2, 3),       # 2-simplex (edge)
    (0, 1, 2),    # 3-simplex (triangle face)
    (0, 1, 3),    # 3-simplex (triangle face)
    (0, 2, 3),    # 3-simplex (triangle face)
    (1, 2, 3)     # 3-simplex (triangle face)
]

# Calculate Betti numbers
betti = incremental_algorithm(simplicial_complexes)
print("Betti Numbers:", betti)



Boundary Matrix:
 [[0 0 0 0 1 1 1 0 0 0 0 0 0 0]
 [0 0 0 0 1 0 0 1 1 0 0 0 0 0]
 [0 0 0 0 0 1 0 1 0 1 0 0 0 0]
 [0 0 0 0 0 0 1 0 1 1 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 1 1 0 0]
 [0 0 0 0 0 0 0 0 0 0 1 0 1 0]
 [0 0 0 0 0 0 0 0 0 0 0 1 1 0]
 [0 0 0 0 0 0 0 0 0 0 1 0 0 1]
 [0 0 0 0 0 0 0 0 0 0 0 1 0 1]
 [0 0 0 0 0 0 0 0 0 0 0 0 1 1]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0]]
Initial Delta:
 [0 0 0 0 1 2 3 2 3 3 7 8 9 9]
Updated column 7 with column 5. New Delta:
 [0 0 0 0 1 2 3 1 3 3 7 8 9 9]
Updated column 7 with column 4. New Delta:
 [0 0 0 0 1 2 3 0 3 3 7 8 9 9]
Updated column 8 with column 6. New Delta:
 [0 0 0 0 1 2 3 0 1 3 7 8 9 9]
Updated column 8 with column 4. New Delta:
 [0 0 0 0 1 2 3 0 0 3 7 8 9 9]
Updated column 9 with column 6. New Delta:
 [0 0 0 0 1 2 3 0 0 2 7 8 9 9]
Updated column 9 with column 5. New Delta:
 [0 0 0 0 1 2 3 0 0 0 7 8 9 9]
Updated column 13 with column 12. New Delta:
 [0 0 0 0 1 

## Application on Torus

In [7]:
# Example triangulation of a torus
simplicial_complexes = [
    (0,),         # 0-simplex (vertex)
    (1,),         # 1-simplex (vertex)
    (2,),         # 1-simplex (vertex)
    (3,),         # 1-simplex (vertex)
    (4,),         # 1-simplex (vertex)
    (5,),         # 1-simplex (vertex)
    (6,),         # 1-simplex (vertex)
    (7,),         # 1-simplex (vertex)
    (8,),         # 1-simplex (vertex)
    (0, 1),       # 2-simplex (edge)
    (1, 2),       # 2-simplex (edge)
    (2, 0),       # 2-simplex (edge)
    (3, 0),       # 2-simplex (edge)
    (3, 1),       # 2-simplex (edge)
    (5, 1),       # 2-simplex (edge)
    (5, 2),       # 2-simplex (edge)
    (2, 6),       # 2-simplex (edge)
    (6, 0),       # 2-simplex (edge)
    (3, 5),       # 2-simplex (edge)
    (5, 6),       # 2-simplex (edge)
    (6, 3),       # 2-simplex (edge)

    (3, 4),       # 2-simplex (edge)
    (4, 5),       # 2-simplex (edge)
    (5, 7),       # 2-simplex (edge)
    (7, 6),       # 2-simplex (edge)
    (6, 8),       # 2-simplex (edge)
    (8, 3),       # 2-simplex (edge)
    (4, 7),       # 2-simplex (edge)
    (7, 8),       # 2-simplex (edge)
    (8, 4),       # 2-simplex (edge)
    (4, 0),       # 2-simplex (edge)
    (0, 7),       # 2-simplex (edge)
    (7, 1),       # 2-simplex (edge)

    (1, 8),       # 2-simplex (edge)
    (8, 2),       # 2-simplex (edge)
    (2, 4),       # 2-simplex (edge)
    
    
    (0, 1, 3),    # 3-simplex (triangle face)
    (1, 3, 5),    # 3-simplex (triangle face)
    (1, 2, 5),    # 3-simplex (triangle face)
    (2, 5, 6),    # 3-simplex (triangle face)
    (2, 0, 6),    # 3-simplex (triangle face)
    (0, 6, 3),    # 3-simplex (triangle face)
    (5, 4, 7),    # 3-simplex (triangle face)
    (3, 4, 5) ,    # 3-simplex (triangle face)

    (5, 6, 7),    # 3-simplex (triangle face)
    (6, 7, 8),    # 3-simplex (triangle face)
    (6, 3, 8),    # 3-simplex (triangle face)
    (3, 8, 4),    # 3-simplex (triangle face)
    (4, 7, 0),    # 3-simplex (triangle face)
    (7, 0, 1),    # 3-simplex (triangle face)
    (7, 8, 1),    # 3-simplex (triangle face)
    (8, 1, 2),     # 3-simplex (triangle face)
    (4, 2, 0),    # 3-simplex (triangle face)
    (8, 4, 2)     # 3-simplex (triangle face)
]

# Calculate Betti numbers
betti = incremental_algorithm(simplicial_complexes)
print("Betti Numbers:", betti)

Boundary Matrix:
 [[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]
Initial Delta:
 [ 0  0  0  0  0  0  0  0  0  1  2  2  3  3  5  5  6  6  5  6  6  4  5  7
  7  8  8  7  8  8  4  7  7  8  8  4 13 18 15 19 17 20 27 22 24 28 26 29
 31 32 33 34 35 35]
Updated column 11 with column 10. New Delta:
 [ 0  0  0  0  0  0  0  0  0  1  2  1  3  3  5  5  6  6  5  6  6  4  5  7
  7  8  8  7  8  8  4  7  7  8  8  4 13 18 15 19 17 20 27 22 24 28 26 29
 31 32 33 34 35 35]
Updated column 11 with column 9. New Delta:
 [ 0  0  0  0  0  0  0  0  0  1  2  0  3  3  5  5  6  6  5  6  6  4  5  7
  7  8  8  7  8  8  4  7  7  8  8  4 13 18 15 19 17 20 27 22 24 28 26 29
 31 32 33 34 35 35]
Updated column 13 with column 12. New Delta:
 [ 0  0  0  0  0  0  0  0  0  1  2  0  3  1  5  5  6  6  5  6  6  4  5  7
  7  8  8  7  8  8  4  7  7  8  8  4 13 18 15 19 17 20 27 22 24 28 26 29
 31 32 33 34 35 35]
Updated column 13 with column 9. New Delta:
 

### Conclusion

The objective of this project was to compute the betti numbers for a give simplices.

